In [1]:
import os
from math import sqrt, atan2, pi
import numpy as np
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

2025-02-24 15:55:32.072266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740441332.089152   49931 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740441332.094590   49931 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-24 15:55:32.112318: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

def calculate_euler_angles(landmarks):
    """Calculate the Euler angles ( yaw, pitch, roll ) for each pose."""
    
    # Get coordinates of key landmarks
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
    right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]

    # Calculate vectors for rotation
    shoulder_to_elbow_left = (left_elbow.x - left_shoulder.x, 
                              left_elbow.y - left_shoulder.y,
                              left_elbow.z - left_shoulder.z)
    
    shoulder_to_hip_right = (right_hip.x - right_shoulder.x,
                             right_hip.y - right_shoulder.y,
                             right_hip.z - right_shoulder.z)

    # Calculate Euler angles
    yaw = atan2(shoulder_to_elbow_left[1], shoulder_to_elbow_left[0])
    pitch = atan2(-shoulder_to_hip_right[2], sqrt(shoulder_to_hip_right[0]**2 + shoulder_to_hip_right[1]**2))
    roll = 0.0

    return yaw, pitch, roll


In [3]:
def write_bvh_file(euler_angles):
    """Write the Euler angles to a BVH file."""
    
    # Define joint hierarchy (simplified for demonstration)
    joints = [
        "Root",
        "LeftArm",
        "RightArm",
        "LeftLeg",
        "RightLeg"
    ]
    
    with open("output.bvh", "w") as f:
        f.write("VERSION 1.3\n")
        f.write("\n")
        
        # Write translation (assuming root is at origin)
        f.write("ROOT Root\n")
        f.write(f"   CHAIN {joints[0]}\n")
        f.write("   TO WorldTransform{\n")
        f.write("      t 0.0 0.0 0.0\n")
        
        # Write Euler angles
        for i, joint in enumerate(joints):
            if i == 0:
                continue
            
            f.write(f"\nJOINT {joint}\n")
            f.write(f"   PARENT {joints[i-1]}\n")
            f.write("   CHAIN " + joints[i] + "\n")
            f.write("   ROTZ YAW\n")
            f.write(f"      r 0.0 {euler_angles[0]} 0.0\n")  # yaw
            f.write("   ROTX PITCH\n")
            f.write(f"      r 0.0 {euler_angles[1]} 0.0\n")  # pitch
            f.write("   ROTY ROLL\n")
            f.write(f"      r 0.0 {euler_angles[2]} 0.0\n")  # roll
            
        f.write("\n")

In [4]:
def main():
    """Main function to process pose data and write BVH file."""
    
    cap = cv2.VideoCapture(0)  # Use webcam
    
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            success, image = cap.read()
            
            if not success:
                break
            
            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            
            if results.pose_landmarks is not None:
                euler_angles = calculate_euler_angles(results.pose_landmarks.landmark)
                
                # Convert radians to degrees (optional, depending on BVH requirements)
                euler_degrees = [angle * 180.0 / pi for angle in euler_angles]
                
                write_bvh_file(euler_degrees)
                
            cv2.imshow('MediaPipe Pose', image)
            
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break
                
    cap.release()

In [5]:
main()

I0000 00:00:1740441226.241369   49536 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1740441226.243669   49786 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.3), renderer: Mesa Intel(R) UHD Graphics (CML GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1740441226.315488   49763 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740441226.350631   49771 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1740441228.646092   49765 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
Only C and default locale supported with the posix collation implementation
Only C and default l

KeyboardInterrupt: 